In [ ]:
import os
import PIL
import pickle
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
path = 'traffic-signs-data.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
with open('traffic-signs-data/train.p', mode='rb') as training_data:
    train = pickle.load(training_data)

with open('traffic-signs-data/test.p', mode='rb') as testing_data:
    test = pickle.load(testing_data)

with open('traffic-signs-data/valid.p', mode='rb') as validation_data:
    valid = pickle.load(validation_data)

In [ ]:
train

In [ ]:
x1 = X_train, y_train = train['features'], train['labels']
x1

In [ ]:
x2 = X_test, y_test = test['features'], test['labels']
x2

In [ ]:
x3 = X_validation, y_validation = valid['features'], valid['labels']
x3

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
X_validation.shape

# Visualização dos dados

In [ ]:
i = 3100
plt.imshow(X_train[i])
y_train[i]

In [ ]:
i = 3200
plt.imshow(X_train[i])
y_train[i]

In [ ]:
i = 3001
plt.imshow(X_test[i])
y_test[i]

In [ ]:
i = 2100
plt.imshow(X_validation[i])
y_validation[i]

# Preparação dos dados

In [ ]:
from sklearn.utils import shuffle

s1 = X_train, y_train = shuffle(X_train, y_train)
s1

In [ ]:
X_train_gray = np.sum(X_train / 3, axis = 3, keepdims=True)
X_train_gray

In [ ]:
X_test_gray = np.sum(X_test / 3, axis = 3, keepdims=True)
X_test_gray

In [ ]:
X_validation_gray = np.sum(X_validation / 3, axis = 3, keepdims=True)
X_validation_gray

In [ ]:
X_train_gray.shape

In [ ]:
X_test_gray.shape

In [ ]:
X_validation_gray.shape

In [ ]:
X_train_gray_norm = (X_train_gray - 128) / 128
X_train_gray_norm

In [ ]:
X_test_gray_norm = (X_test_gray - 128) / 128
X_test_gray_norm

In [ ]:
X_validation_gray_norm = (X_validation_gray - 128) / 128
X_validation_gray_norm

In [ ]:
np.max(X_train_gray_norm)

In [ ]:
np.min(X_train_gray_norm)

In [ ]:
i = 60
plt.imshow(X_train_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_train[i])
plt.figure()
plt.imshow(X_train_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
i = 350
plt.imshow(X_test_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_test[i])
plt.figure()
plt.imshow(X_test_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
i = 610
plt.imshow(X_test_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_test[i])
plt.figure()
plt.imshow(X_test_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
i = 500
plt.imshow(X_validation_gray[i].squeeze(), cmap='gray')
plt.figure()
plt.imshow(X_validation[i])
plt.figure()
plt.imshow(X_validation_gray_norm[i].squeeze(), cmap = 'gray')

In [ ]:
from tensorflow.keras import layers, models

LeNet = models.Sequential()
LeNet.add(layers.Conv2D(6, (5,5), activation = 'relu', input_shape = (32, 32, 1)))
LeNet.add(layers.AveragePooling2D())
LeNet.add(layers.Conv2D(16, (5,5), activation = 'relu'))
LeNet.add(layers.AveragePooling2D())
LeNet.add(layers.Flatten())
LeNet.add(layers.Dense(120, activation='relu'))
LeNet.add(layers.Dense(84, activation='relu'))
LeNet.add(layers.Dense(43, activation='softmax'))

LeNet.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
LeNet.summary()

In [ ]:
history = LeNet.fit(X_train_gray_norm, 
                    y_train,
                    batch_size = 500,
                    epochs = 50,
                    validation_data = (X_validation_gray_norm, y_validation))

# Avaliação do modelo

In [ ]:
score = LeNet.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy: {}'.format(score[1]))

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
accuracy

In [ ]:
val_accuracy = history.history['val_accuracy']
val_accuracy

In [ ]:
loss = history.history['loss']
loss

In [ ]:
val_loss = history.history['val_loss']
val_loss

In [ ]:
plt.figure(figsize = (25, 15))

plt.plot(history.history["accuracy"], label='Training Accuracy')
plt.plot(history.history["val_accuracy"], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
plt.figure(figsize = (25, 15))

plt.plot(history.history["loss"], label='Training loss')
plt.plot(history.history["val_loss"], label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

In [ ]:
predicted_classes = LeNet.predict_classes(X_test_gray_norm)
y_true = y_test
predicted_classes

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, predicted_classes)
cm

In [ ]:
plt.figure(figsize = (25, 15))
sns.heatmap(cm, annot = True);

In [ ]:
L = 7
W = 7

fig, axes = plt.subplots(L, W, figsize = (15, 15))
axes = axes.ravel()

for i in np.arange(0, L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(predicted_classes[i], y_true[i]), fontsize = 15)
    axes[i].axis('off')

plt.subplots_adjust(wspace = 1)